## High Freq

### 410 Split Array Largest Sum

Given an array which consists of non-negative integers and an integer m, you can split the array into m non-empty continuous subarrays. Write an algorithm to minimize the largest sum among these m subarrays.

```
Note:
If n is the length of array, assume the following constraints are satisfied:

1 ≤ n ≤ 1000
1 ≤ m ≤ min(50, n)
Examples:

Input:
nums = [7,2,5,10,8]
m = 2

Output:
18

Explanation:
There are four ways to split nums into two subarrays.
The best way is to split it into [7,2,5] and [10,8],
where the largest sum among the two subarrays is only 18.
```

In [ ]:
# dp o(mn^2)
class Solution: 
    def splitArray(self, nums: List[int], m: int) -> int:
        n = len(nums)
        cums = [0]
        for i in range(n):
            cums.append(cums[i]+nums[i])
        
        dp = [[float('inf') for j in range(n+1)]
                            for i in range(m+1)]
        # stands for m cut in nums[:n]
        dp[0][0] = 0 
        #O(mn^2)
        for i in range(1, m+1):
            for j in range(1, n+1):
                for k in range(i-1, j):
                    # val = max(dp[i-1][k], sum(nums[k:j]))
                    # dp[i-1][k]表示数组中前k个数字分成i-1组所能得到的最小的各个子数组中最大值
                    val = max(dp[i-1][k], cums[j]-cums[k])
                    dp[i][j] = min(dp[i][j], val)
        
        return dp[m][n]

In [5]:
#O(n*32)
class Solution:
    def splitArray(self, nums: list, m: int) -> int:
        
        def check(mid):#O(n)
            # can we make at-most m sub arrays with maxium sum at most mid
            cuts, cur_sum = 0, 0 
            for x in nums:
                cur_sum += x 
                if cur_sum > mid:
                    cuts, cur_sum = cuts+1, x
            subs = cuts + 1 
            return (subs <= m)
        
        #O(n)
        start, end = max(nums), sum(nums)
        
        #O(32) if int is bounded to 2^32
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if check(mid):
                end = mid
            else:
                start = mid 
        
        if check(start):
            return start
        return end

### 975 Odd Even Jump(Monotone Stack)

You are given an integer array A.  From some starting index, you can make a series of jumps.  The (1st, 3rd, 5th, ...) jumps in the series are called odd numbered jumps, and the (2nd, 4th, 6th, ...) jumps in the series are called even numbered jumps.

You may from index i jump forward to index j (with i < j) in the following way:

During odd numbered jumps (ie. jumps 1, 3, 5, ...), you jump to the index j such that A[i] <= A[j] and A[j] is the smallest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
During even numbered jumps (ie. jumps 2, 4, 6, ...), you jump to the index j such that A[i] >= A[j] and A[j] is the largest possible value.  If there are multiple such indexes j, you can only jump to the smallest such index j.
(It may be the case that for some index i, there are no legal jumps.)
A starting index is good if, starting from that index, you can reach the end of the array (index A.length - 1) by jumping some number of times (possibly 0 or more than once.)

Return the number of good starting indexes.

```
Example 1:

Input: [10,13,12,14,15]
Output: 2
Explanation: 
From starting index i = 0, we can jump to i = 2 (since A[2] is the smallest among A[1], A[2], A[3], A[4] that is greater or equal to A[0]), then we can't jump any more.
From starting index i = 1 and i = 2, we can jump to i = 3, then we can't jump any more.
From starting index i = 3, we can jump to i = 4, so we've reached the end.
From starting index i = 4, we've reached the end already.
In total, there are 2 different starting indexes (i = 3, i = 4) where we can reach the end with some number of jumps.
```

In [ ]:
class Solution:
    def oddEvenJumps(self, A: List[int]) -> int:
        n = len(A)
        next_higher, next_lower = [0]*n, [0]*n
        
        stack = []
        
        for a, i in sorted([a,i] for i, a in enumerate(A)):
            while stack and stack[-1] < i:
                next_higher[stack.pop()] = i
            stack.append(i)
            
        stack = []
        
        for a, i in sorted([-a,i] for i, a in enumerate(A)):
            while stack and stack[-1] < i:
                next_lower[stack.pop()] = i 
            stack.append(i)
        
        higher, lower = [0]*n, [0]*n
        higher[-1] = lower[-1] = 1 
        
        for i in range(n-1)[::-1]:
            higher[i] = lower[next_higher[i]]
            lower[i] = higher[next_lower[i]]
        
        return sum(higher)

#### 单调栈小结

所谓的单调栈 Monotone Stack，就是栈内元素都是单调递增或者单调递减的，有时候需要严格的单调递增或递减，根据题目的具体情况来看吧。举个生动的例子来说明吧：比如有一天，某家店在发 free food，很多人在排队，于是你也赶过去凑热闹。但是由于来晚了，队伍已经很长了，想着不然就插个队啥的。但发现排在队伍最前面的都是一些有纹身的大佬，惹不起，只能赞美道，小猪佩奇身上纹，来世还做社会人。于是往队伍后面走，发现是一群小屁孩，直接全部撵走，然后排在了社会大佬们的后面。那么这就是一个单调递减的栈，按实力递减。由于栈元素是后进先出的，所以上面的例子正确的检查顺序应该是从队尾往前遍历，小屁孩都撵走，直到遇到大佬停止，然后排在大佬后面（假设这个队列已经事先按实力递减排好了）。

1.单调栈里的元素具有单调性

2.元素加入栈前，会在栈顶端把破坏栈单调性的元素都删除

3.使用单调栈（单增）可以找到元素向左遍历第一个比他小的元素，也可以找到元素向左遍历第一个比他大的元素。

### Next Exceed

给一个数组，返回一个大小相同的数组。返回的数组的第i个位置的值应当是，对于原数组中的第i个元素，至少往右走多少步，才能遇到一个比自己大的元素（如果之后没有比自己大的元素，或者已经是最后一个元素，则在返回数组的对应位置放上-1）。

简单的例子：

input: 5,3,1,2,4

return: -1 3 1 1 -1

In [3]:
def nextExceed(A:list):
    n = len(A)
    res = [-1] * n
    stack = []
    for i in range(n):
        while stack and A[i] > A[stack[-1]]:
        #前面比我弱的都踢了，
            res[stack[-1]] = i - stack[-1]
            stack.pop()
        stack.append(i)
    print(res)

nextExceed([5,3,1,2,4])            

[-1, 3, 1, 1, -1]


### Largest Rectangle in Histogram


Given n non-negative integers representing the histogram's bar height where the width of each bar is 1, find the area of largest rectangle in the histogram.

 
![](https://assets.leetcode.com/uploads/2018/10/12/histogram_area.png)

Above is a histogram where width of each bar is 1, given height = [2,1,5,6,2,3].

 


The largest rectangle is shown in the shaded area, which has area = 10 unit.

In [5]:
class Solution:
    def largestRectangleArea(self, heights: list) -> int:
        heights.append(0)
        n = len(heights)
        stack = [-1] 
        ans = 0 
        for i in range(n):
            while heights[i] < heights[stack[-1]]: 
                # 单增栈，前面比我大的都T了，我就是前面比我大的这些里面next smaller
                # 栈顶元素高度是矩形的高度
                h = heights[stack[-1]]
                stack.pop() # 栈顶是矩形位置，必须在这里pop，因为前面可能有gap
                ans = max(ans, h * (i - stack[-1] - 1))        
            stack.append(i)
        heights.pop()
        return ans
Solution().largestRectangleArea([4, 1, 3, 2, 2])

6

### Maximal Rectangle
Given a 2D binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.

```
Example:

Input:
[
  ["1","0","1","0","0"],
  ["1","0","1","1","1"],
  ["1","1","1","1","1"],
  ["1","0","0","1","0"]
]
Output: 6
```

In [ ]:
class Solution:
    def maximalRectangle(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        ans, heights = 0, [0] * len(matrix[0])
        for row in matrix:
            for index, num in enumerate(row):
                heights[index] = heights[index] + 1 if num == '1' else 0
            
            ans = max(ans, self.maxRect(heights))
        return ans      
          
    def maxRect(self, A:list):
        A.append(0)
        stack = [-1]
        ans = 0
        for i in range(len(A)):
            while A[i] < A[stack[-1]]:
                h = A[stack.pop()]
                ans = max(ans, h*(i - stack[-1] - 1))
            stack.append(i)
        A.pop()
        return ans
                

### Trapping Rain Water


Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining.

![](https://assets.leetcode.com/uploads/2018/10/22/rainwatertrap.png)

The above elevation map is represented by array [0,1,0,2,1,0,1,3,2,1,2,1]. In this case, 6 units of rain water (blue section) are being trapped. Thanks Marcos for contributing this image!

Example:

Input: [0,1,0,2,1,0,1,3,2,1,2,1]

Output: 6

In [ ]:
class Solution:
    def maximalRectangle(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        ans, heights = 0, [0] * len(matrix[0])
        for row in matrix:
            for index, num in enumerate(row):
                heights[index] = heights[index] + 1 if num == '1' else 0
            ans = max(ans, self.maxRect(heights))
        return ans      
          
    def maxRect(self, A:list):
        A.append(0)
        stack = [-1]
        ans = 0
        for i in range(len(A)):
            while A[i] < A[stack[-1]]:
                h = A[stack.pop()]
                ans = max(ans, h*(i - stack[-1] - 1))
            stack.append(i)
        A.pop()
        return ans

### Campus Bikes
Medium
On a campus represented as a 2D grid, there are N workers and M bikes, with N <= M. Each worker and bike is a 2D coordinate on this grid.

Our goal is to assign a bike to each worker. Among the available bikes and workers, we choose the (worker, bike) pair with the shortest Manhattan distance between each other, and assign the bike to that worker. (If there are multiple (worker, bike) pairs with the same shortest Manhattan distance, we choose the pair with the smallest worker index; if there are multiple ways to do that, we choose the pair with the smallest bike index). We repeat this process until there are no available workers.

The Manhattan distance between two points p1 and p2 is Manhattan(p1, p2) = |p1.x - p2.x| + |p1.y - p2.y|.

Return a vector ans of length N, where ans[i] is the index (0-indexed) of the bike that the i-th worker is assigned to.

```
Input: workers = [[0,0],[2,1]], bikes = [[1,2],[3,3]]
Output: [1,0]
Explanation: 
Worker 1 grabs Bike 0 as they are closest (without ties), and Worker 0 is assigned Bike 1. So the output is [1, 0].
```

In [ ]:
class Solution:
    def assignBikes(self, workers: List[List[int]], bikes: List[List[int]]) -> List[int]:
        ans = [-1] * len(workers)
        used = set()
        dists = []
        
        for i in range(len(workers)):
            for j in range(len(bikes)):
                dis = abs(workers[i][0] - bikes[j][0]) + abs(workers[i][1] - bikes[j][1])
                dists.append((dis, i, j))      
        
        
        for dis, w, b in sorted(dists):
            if ans[w] == -1 and b not in used:
                ans[w] = b
                used.add(b)
        return ans

### Campus Bikes II
Medium

On a campus represented as a 2D grid, there are N workers and M bikes, with N <= M. Each worker and bike is a 2D coordinate on this grid.

We assign one unique bike to each worker so that the sum of the Manhattan distances between each worker and their assigned bike is minimized.

The Manhattan distance between two points p1 and p2 is Manhattan(p1, p2) = |p1.x - p2.x| + |p1.y - p2.y|.

Return the minimum possible sum of Manhattan distances between each worker and their assigned bike.

 
```
Example 1:

Input: workers = [[0,0],[2,1]], bikes = [[1,2],[3,3]]
Output: 6
Explanation: 
We assign bike 0 to worker 0, bike 1 to worker 1. The Manhattan distance of both assignments is 3, so the output is 6.
Example 2:



Input: workers = [[0,0],[1,1],[2,0]], bikes = [[1,0],[2,2],[2,1]]
Output: 4
Explanation: 
We first assign bike 0 to worker 0, then assign bike 1 to worker 1 or worker 2, bike 2 to worker 2 or worker 1. Both assignments lead to sum of the Manhattan distances as 4.
 

Note:

0 <= workers[i][0], workers[i][1], bikes[i][0], bikes[i][1] < 1000
All worker and bike locations are distinct.
1 <= workers.length <= bikes.length <= 10
```

###  Count Complete Tree Nodes

Given a complete binary tree, count the number of nodes.

Note:

Definition of a complete binary tree from Wikipedia:
In a complete binary tree every level, except possibly the last, is completely filled, and all nodes in the last level are as far left as possible. It can have between 1 and 2h nodes inclusive at the last level h.
```
Example:

Input: 
    1
   / \
  2   3
 / \  /
4  5 6

Output: 6
```

In [ ]:
 class Solution:#O(lognlogn)
        def countNodes(self, root):
            if not root:
                return 0
            leftDepth = self.getDepth(root.left)
            rightDepth = self.getDepth(root.right)
            if leftDepth == rightDepth:
                return pow(2, leftDepth) + self.countNodes(root.right)
            else:
                return pow(2, rightDepth) + self.countNodes(root.left)
    
        def getDepth(self, root):
            if not root:
                return 0
            return 1 + self.getDepth(root.left)

### Guess the Word

This problem is an interactive problem new to the LeetCode platform.

We are given a word list of unique words, each word is 6 letters long, and one word in this list is chosen as secret.

You may call master.guess(word) to guess a word.  The guessed word should have type string and must be from the original list with 6 lowercase letters.

This function returns an integer type, representing the number of exact matches (value and position) of your guess to the secret word.  Also, if your guess is not in the given wordlist, it will return -1 instead.

For each test case, you have 10 guesses to guess the word. At the end of any number of calls, if you have made 10 or less calls to master.guess and at least one of these guesses was the secret, you pass the testcase.

Besides the example test case below, there will be 5 additional test cases, each with 100 words in the word list.  The letters of each word in those testcases were chosen independently at random from 'a' to 'z', such that every word in the given word lists is unique.
```
Example 1:
Input: secret = "acckzz", wordlist = ["acckzz","ccbazz","eiowzz","abcczz"]

Explanation:

master.guess("aaaaaa") returns -1, because "aaaaaa" is not in wordlist.
master.guess("acckzz") returns 6, because "acckzz" is secret and has all 6 matches.
master.guess("ccbazz") returns 3, because "ccbazz" has 3 matches.
master.guess("eiowzz") returns 2, because "eiowzz" has 2 matches.
master.guess("abcczz") returns 4, because "abcczz" has 4 matches.

We made 5 calls to master.guess and one of them was the secret, so we pass the test case.
```

In [ ]:
class Solution(object):
    def findSecretWord(self, wordlist, master):
        def pair_matches(a, b):         # count the number of matching characters
            return sum(c1 == c2 for c1, c2 in zip(a, b))

        def most_overlap_word():
            counts = [[0 for _ in range(26)] for _ in range(6)]     # counts[i][j] is nb of words with char j at index i
            for word in candidates:
                for i, c in enumerate(word):
                    counts[i][ord(c) - ord("a")] += 1

            best_score = 0
            for word in candidates:
                score = 0
                for i, c in enumerate(word):
                    score += counts[i][ord(c) - ord("a")]           # all words with same chars in same positions
                if score > best_score:
                    best_score = score
                    best_word = word

            return best_word

        candidates = wordlist[:]        # all remaining candidates, initially all words
        while candidates:

            s = most_overlap_word()     # guess the word that overlaps with most others
            matches = master.guess(s)

            if matches == 6:
                return

            candidates = [w for w in candidates if pair_matches(s, w) == matches]   # filter words with same matches

### Unique Email Addresses

Every email consists of a local name and a domain name, separated by the @ sign.

For example, in alice@leetcode.com, alice is the local name, and leetcode.com is the domain name.

Besides lowercase letters, these emails may contain '.'s or '+'s.

If you add periods ('.') between some characters in the local name part of an email address, mail sent there will be forwarded to the same address without dots in the local name.  For example, "alice.z@leetcode.com" and "alicez@leetcode.com" forward to the same email address.  (Note that this rule does not apply for domain names.)

If you add a plus ('+') in the local name, everything after the first plus sign will be ignored. This allows certain emails to be filtered, for example m.y+name@email.com will be forwarded to my@email.com.  (Again, this rule does not apply for domain names.)

It is possible to use both of these rules at the same time.

Given a list of emails, we send one email to each address in the list.  How many different addresses actually receive mails? 

 
```
Example 1:

Input: ["test.email+alex@leetcode.com","test.e.mail+bob.cathy@leetcode.com","testemail+david@lee.tcode.com"]
Output: 2
Explanation: "testemail@leetcode.com" and "testemail@lee.tcode.com" actually receive mails
 

Note:

1 <= emails[i].length <= 100
1 <= emails.length <= 100
Each emails[i] contains exactly one '@' character.
All local and domain names are non-empty.
Local names do not start with a '+' character.
```

### Cracking the Safe
Hard
There is a box protected by a password. The password is a sequence of n digits where each digit can be one of the first k digits 0, 1, ..., k-1.

While entering a password, the last n digits entered will automatically be matched against the correct password.

For example, assuming the correct password is "345", if you type "012345", the box will open because the correct password matches the suffix of the entered password.

Return any password of minimum length that is guaranteed to open the box at some point of entering it.

 
```
Example 1:

Input: n = 1, k = 2
Output: "01"
Note: "10" will be accepted too.
Example 2:

Input: n = 2, k = 2
Output: "00110"
Note: "01100", "10011", "11001" will be accepted too.
```

In [ ]:
class Solution:
    def crackSafe(self, n: int, k: int) -> str:
        def dfs(curr, counted, total):
            if len(counted) == total:
                return curr
            
            for i in range(k):
                tmp = curr[-(n-1):] + str(i) if n!=1 else str(i)
                if tmp not in counted:
                    counted.add(tmp)
                    res = dfs(curr + str(i), counted, total)
                    if res:
                        return res
                    counted.remove(tmp)
        return dfs('0'*n, set(['0'*n]), k**n)

### Union Find 小结

### Most Stones Removed with Same Row or Column

On a 2D plane, we place stones at some integer coordinate points.  Each coordinate point may have at most one stone.

Now, a move consists of removing a stone that shares a column or row with another stone on the grid.

What is the largest possible number of moves we can make?

 
```
Example 1:

Input: stones = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
Output: 5
Example 2:

Input: stones = [[0,0],[0,2],[1,1],[2,0],[2,2]]
Output: 3
Example 3:

Input: stones = [[0,0]]
Output: 0
 

Note:

1 <= stones.length <= 1000
0 <= stones[i][j] < 10000
```
如果两个石头在同行或者同列，两个石头就是连接的。连在一起的石头，可以组成一个连通图。每一个连通图至少会剩下1个石头。
所以我们希望存在一种思路，每个连通图都只剩下1个石头。
这样这题就转化成了数岛屿的问题。

In [ ]:
from collections import defaultdict
class Solution:
    def removeStones(self, stones: List[List[int]]) -> int:
        cols = defaultdict(set)
        rows = defaultdict(set)
        for x, y in stones:
            cols[y].add(x)
            rows[x].add(y)
        
        def dfsRow(i):
            seenR.add(i)
            for j in rows[i]:
                if j not in seenC:
                    dfsCol(j)
        
        def dfsCol(j):
            seenC.add(j)
            for i in cols[j]:
                if i not in seenR:
                    dfsRow(i)
        
        seenC, seenR = set(), set()
        islands = 0 
        for i, j in stones:
            if i not in seenR:
                islands += 1
                dfsRow(i)
                dfsCol(j)
        return len(stones) - islands

In [ ]:
from collections import defaultdict
class Solution:
    def removeStones(self, stones: List[List[int]]) -> int:
        UF = {}
        def find(x):
            if x != UF[x]:
                UF[x] = find(UF[x])
            return UF[x]
        
        def union(x, y):
            UF.setdefault(x,x)
            UF.setdefault(y,y)
            UF[find(x)] = UF[find(y)]
        
        for x, y in stones:
            union(x, ~y)
        
        return len(stones) - len({find(x) for x in UF})
        # UF 里面，1->2->-3, 所以不能直接set(UF.values())

In [16]:
divmod(5,2)

(2, 1)

### License Key Formatting
Easy
You are given a license key represented as a string S which consists only alphanumeric character and dashes. The string is separated into N+1 groups by N dashes.

Given a number K, we would want to reformat the strings such that each group contains exactly K characters, except for the first group which could be shorter than K, but still must contain at least one character. Furthermore, there must be a dash inserted between two groups and all lowercase letters should be converted to uppercase.

Given a non-empty string S and a number K, format the string according to the rules described above.
```
Example 1:
Input: S = "5F3Z-2e-9-w", K = 4

Output: "5F3Z-2E9W"

Explanation: The string S has been split into two parts, each part has 4 characters.
Note that the two extra dashes are not needed and can be removed.
Example 2:
Input: S = "2-5g-3-J", K = 2

Output: "2-5G-3J"

Explanation: The string S has been split into three parts, each part has 2 characters except the first part as it could be shorter as mentioned above.
Note:
The length of string S will not exceed 12,000, and K is a positive integer.
String S consists only of alphanumerical characters (a-z and/or A-Z and/or 0-9) and dashes(-).
String S is non-empty.
```

In [ ]:
class Solution:
    def licenseKeyFormatting(self, S: str, K: int) -> str:
        S = S.replace('-','').upper()
        div, res = divmod(len(S), K)
        ans = [S[:res]] if res else []
        S = S[res:]
        for i in range(div):
            ans.append(S[i*K:(i+1)*K])
        return '-'.join(ans)

### Robot Room Cleaner
Hard
Given a robot cleaner in a room modeled as a grid.

Each cell in the grid can be empty or blocked.

The robot cleaner with 4 given APIs can move forward, turn left or turn right. Each turn it made is 90 degrees.

When it tries to move into a blocked cell, its bumper sensor detects the obstacle and it stays on the current cell.

Design an algorithm to clean the entire room using only the 4 given APIs shown below.

interface Robot {
  // returns true if next cell is open and robot moves into the cell.
  // returns false if next cell is obstacle and robot stays on the current cell.
  boolean move();

  // Robot will stay on the same cell after calling turnLeft/turnRight.
  // Each turn will be 90 degrees.
  void turnLeft();
  void turnRight();

  // Clean the current cell.
  void clean();
}

```
Example:

Input:
room = [
  [1,1,1,1,1,0,1,1],
  [1,1,1,1,1,0,1,1],
  [1,0,1,1,1,1,1,1],
  [0,0,0,1,0,0,0,0],
  [1,1,1,1,1,1,1,1]
],
row = 1,
col = 3

Explanation:
All grids in the room are marked by either 0 or 1.
0 means the cell is blocked, while 1 means the cell is accessible.
The robot initially starts at the position of row=1, col=3.
From the top left corner, its position is one row below and three columns right.
Notes:

The input is only given to initialize the room and the robot's position internally. You must solve this problem "blindfolded". In other words, you must control the robot using only the mentioned 4 APIs, without knowing the room layout and the initial robot's position.
The robot's initial position will always be in an accessible cell.
The initial direction of the robot will be facing up.
All accessible cells are connected, which means the all cells marked as 1 will be accessible by the robot.
Assume all four edges of the grid are all surrounded by wall.
```

In [ ]:
class Solution(object):
    def cleanRoom(self, robot):
        """
        :type robot: Robot
        :rtype: None
        """
        self.dfs(robot, 0, 0, 0, 1, set())
    
    def dfs(self, robot, x, y, direction_x, direction_y, visited):
        robot.clean()
        visited.add((x, y))
        
        for k in range(4):
            neighbor_x = x + direction_x
            neighbor_y = y + direction_y
            if (neighbor_x, neighbor_y) not in visited and robot.move():
                self.dfs(robot, neighbor_x, neighbor_y, direction_x, direction_y, visited)
                robot.turnLeft()
                robot.turnLeft()
                robot.move()
                robot.turnLeft()
                robot.turnLeft()
            robot.turnLeft()
            direction_x, direction_y = -direction_y, direction_x   

###  Backspace String Compare
Easy

Favorite

Share
Given two strings S and T, return if they are equal when both are typed into empty text editors. # means a backspace character.
```
Example 1:

Input: S = "ab#c", T = "ad#c"
Output: true
Explanation: Both S and T become "ac".
Example 2:

Input: S = "ab##", T = "c#d#"
Output: true
Explanation: Both S and T become "".
Example 3:

Input: S = "a##c", T = "#a#c"
Output: true
Explanation: Both S and T become "c".
Example 4:

Input: S = "a#c", T = "b"
Output: false
Explanation: S becomes "c" while T becomes "b".
Note:

1 <= S.length <= 200
1 <= T.length <= 200
S and T only contain lowercase letters and '#' characters.
Follow up:

Can you solve it in O(N) time and O(1) space?
```

### Cat and Mouse
Hard
A game on an undirected graph is played by two players, Mouse and Cat, who alternate turns.

The graph is given as follows: graph[a] is a list of all nodes b such that ab is an edge of the graph.

Mouse starts at node 1 and goes first, Cat starts at node 2 and goes second, and there is a Hole at node 0.

During each player's turn, they must travel along one edge of the graph that meets where they are.  For example, if the Mouse is at node 1, it must travel to any node in graph[1].

Additionally, it is not allowed for the Cat to travel to the Hole (node 0.)

Then, the game can end in 3 ways:

If ever the Cat occupies the same node as the Mouse, the Cat wins.
If ever the Mouse reaches the Hole, the Mouse wins.
If ever a position is repeated (ie. the players are in the same position as a previous turn, and it is the same player's turn to move), the game is a draw.
Given a graph, and assuming both players play optimally, return 1 if the game is won by Mouse, 2 if the game is won by Cat, and 0 if the game is a draw.

 
```
Example 1:

Input: [[2,5],[3],[0,4,5],[1,4,5],[2,3],[0,2,3]]
Output: 0
Explanation:
4---3---1
|   |
2---5
 \ /
  0
 

Note:

3 <= graph.length <= 50
It is guaranteed that graph[1] is non-empty.
It is guaranteed that graph[2] contains a non-zero element. 
```

## 1~150

### 001 Two Sum

In [ ]:
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        su = {}
        for i, num in enumerate(nums):
            x = target - num
            if x in su:
                return sorted([i, su[x]])            
            su[num] = i
            
        return [-1,-1]

###  003 Longest Substring Without Repeating Characters
```
Input: "abcabcbb"
Output: 3 
Explanation: The answer is "abc", with the length of 3. 
```

In [ ]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        i, j, ans, n = 0, 0, 0, len(s)
        chset = set()
        for i in range(n):
            while j < n and s[j] not in chset:
                chset.add(s[j])
                j += 1 
            ans = max(ans, j-i)
            chset.remove(s[i])
        return ans

### 004 Median of Two Sorted Arrays

In [ ]:
class Solution:
    def findMedianSortedArrays(self, A: List[int], B: List[int]) -> float:
        n = len(A) + len(B)
        if n % 2 == 1:
            return self.findKth(A, B, n//2 + 1)
        else:
            smaller = self.findKth(A, B, n//2)
            bigger = self.findKth(A, B, n//2 + 1)
            return (smaller + bigger) / 2.0

    def findKth(self, A, B, k):
        if len(A) == 0:
            return B[k-1]
        if len(B) == 0:
            return A[k-1]
        if k == 1:
            return min(A[0], B[0])
        
        a = A[k//2-1] if len(A) >= k//2 else None
        b = B[k//2-1] if len(B) >= k//2 else None
        
        if b is None or (a is not None and a < b):
            return self.findKth(A[k//2:], B, k - k//2)
        return self.findKth(A, B[k//2:], k-k//2)

### 005 Longest Palindromic Substring

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        n, ans = len(s), ''
        
        for i in range(n):
            #odd
            start, end = i, i
            while start >= 0 and end < n and s[start] == s[end]:
                start -= 1
                end += 1
            ans = s[start+1:end] if end - start - 1 > len(ans) else ans
            
            #even
            start, end = i, i+1
            while start >= 0 and end < n and s[start] == s[end]:
                start -= 1
                end += 1
            ans = s[start+1:end] if end - start - 1 > len(ans) else ans
        return ans

###  006 ZigZag Conversion

In [ ]:
class Solution:
    def convert(self, s: str, numRows: int) -> str:
        result = [[] for _ in range(numRows)]
        res, i, flag, n = '', 0, 0, len(s)
        while i < n:
            if flag == 0:
                for j in range(len(result)):
                    if i < len(s):
                        result[j].append(s[i])
                        i += 1
                flag = 1 
            else:
                for j in range(len(result)-2, 0, -1):
                    if i < len(s):
                        result[j].append(s[i])
                        i += 1
                flag = 0
        for words in result:
            for word in words:
                res += word
        return res 
                    

### 007 Reverse Integer

In [ ]:
class Solution:
    def reverse(self, x: int) -> int:
        sign = 1 if x >= 0 else -1
        ans = sign * xx # remove -
        ans = int(str(ans)[::-1])
        ans = sign * ans
        
        if -2**31 <= ans <= 2**31 -1:
            return ans
        return 0
    

### 008 String to Integer (atoi)

In [ ]:
MAX_INT = 2 ** 31 - 1
MIN_INT = -2 ** 31
class Solution:
    def myAtoi(self, stri: str) -> int:
        start = 0
        n = len(stri)

        # skip white space
        while start < n and stri[start] == ' ':
            start += 1

        if start >= n:
            return 0
        
        
        # deal with +, -, non-digit.
        sign = 1
        nums = map(str, range(10))
        start_set = {'-', '+'} | set(nums)
        if stri[start] not in start_set:
            return 0

        elif stri[start] == '-':
            start += 1
            sign = -1
        elif stri[start] == '+':
            start += 1
            sign = 1

        end = start

        while end < n and stri[end].isdigit():
            end += 1

        if start >= end:
            return 0

        ans = sign * int(stri[start:end])

        if ans < MIN_INT:
            return MIN_INT
        if ans > MAX_INT:
            return MAX_INT

        return ans

### 010 Regular Expression Matching

In [ ]:
class Solution(object):
    def isMatch1(self, s, p):
        if not p:
            return not s

        first_match = bool(s) and p[0] in {s[0], '.'}

        if len(p) >= 2 and p[1] == '*':
            return self.isMatch(s, p[2:]) or first_match and self.isMatch(s[1:], p)
        else:
            return first_match and self.isMatch(s[1:], p[1:])

    def isMatch(self, s, p):
        # f: s[i:] and p[j:] match?
        f = [[False for j in range(len(p)+1)]
                    for i in range(len(s)+1)]

        f[-1][-1] = True
        for i in range(len(s), -1, -1):
            for j in range(len(p)-1, -1, -1):
                first_match = i < len(s) and p[j] in {s[i], '.'}
                if j + 1 < len(p) and p[j+1] == '*':
                    f[i][j] = f[i][j+2] or first_match and f[i+1][j]
                else:
                    f[i][j] = first_match and f[i+1][j+1]

        return f[0][0]

if __name__ == "_main__":
    s = Solution()
    print(s.isMatch('aab','c*a*b'))
    print(s.isMatch('aa','a'))
    print(s.isMatch('aa','a*'))
    print(s.isMatch('ab','.*'))
    print(s.isMatch('mississippi','mis*is*p*'))

### 011 Container With Most Water

In [ ]:
class Solution(object):
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        start, end = 0, len(height)-1
        ans = 0 
        while start < end:
            if height[start] < height[end]:                
                ans = max(ans, (end - start) * height[start])
                start += 1
            else:
                ans = max(ans, (end - start) * height[end])
                end -= 1
        
        return ans 

### 012 Integer to Roman

In [ ]:
class Solution(object):
    def intToRoman(self, num):
        M = ["", "M", "MM", "MMM"];
        C = ["", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM"];
        X = ["", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC"];
        I = ["", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX"];
        return M[num//1000] + C[(num%1000)//100] + X[(num%100)//10] + I[num%10];

###  013 Roman to Integer

In [ ]:
class Solution:
    def romanToInt(self, s: str) -> int:
        ROI = {'I':1,
               'V':5,
               'X':10,
               'L':50,
               'C':100,
               'D':500,
               'M':1000}
        
        index = len(s) - 2 
        rum = ROI[s[-1]]
        while index >= 0:
            ch1, ch2 = s[index], s[index+1]
            if ROI[ch1] < ROI[ch2]:
                rum -= ROI[ch1]
            else:
                rum += ROI[ch1]
            index -= 1
        return rum

### 015 3Sum

In [ ]:
class Solution:
    def threeSum(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        res = []
        nums.sort()
        
        def dfs(start, tmp):
            if len(tmp) > 3 or sum(tmp) > 0:
                return 
            if len(tmp) == 3 and sum(tmp) == 0:
                res.append(tmp[:])
                return
            
            for i in range(start, len(nums)):
                if i == start or nums[i] != nums[i-1]:
                    dfs(i+1, tmp + [nums[i]])
        
        dfs(0, [])
        return res

    def threeSum2(self, nums):
        nums.sort()
        res = []
        for i in range(len(nums)-2):
            if i > 0 and nums[i] == nums[i-1]:
                continue
            if nums[i] > 0: # good enhancement.
                break
            start, end = i+1, len(nums) - 1
            while start < end:
                target = nums[i] + nums[start] + nums[end]
                if target == 0:
                    res.append([nums[i], nums[start], nums[end]])
                    start += 1
                    while start < end and nums[i] + nums[start] + nums[end] == 0:
                        start += 1
                elif target < 0:
                    start += 1
                else:
                    end -= 1
                

        return res 
    def threeSum3(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        nums.sort()
        ans = []
        for i in range(0, len(nums)):
            if i > 0 and nums[i] == nums[i-1]:
                continue
            if nums[i] > 0:
                break
            
            lo = i + 1 
            hi = len(nums) - 1 
            
            while lo < hi:
                target = nums[i] + nums[lo] + nums[hi]
                if target > 0:
                    hi -= 1
                elif target < 0:
                    lo += 1
                else:
                    ans.append([nums[i], nums[lo], nums[hi]])
                    #lo += 1
                    while lo < hi and nums[i] + nums[lo] + nums[hi] == 0:
                        lo += 1
        return ans
        
if __name__ == "?_main__":
    s = Solution()
    print(s.threeSum([-1, 0, 1, 2, -1, -4]))

### 016 3Sum Closest

In [ ]:
class Solution(object):
    def threeSumClosest(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: int
        """
        nums.sort()
        n = len(nums)
        ans = 2**32
        for i in range(n - 2):
            start = i+1
            end = n-1
            while start < end:
                cum = nums[i] + nums[start] + nums[end]
                ans = cum if abs(cum - target) < abs(ans - target) else ans
                if cum < target:
                    start += 1
                elif cum == target:
                    return cum
                else:
                    end -= 1
            
        return ans 

### 017 Letter Combinations of a Phone Number

In [ ]:
class Solution:
    def letterCombinations(self, digits: str) -> List[str]:
        if len(digits) == 0:
            return []
        
        ht = {'2':{'a','b','c'},
              '3':{'d','e','f'},
              '4':{'g','h','i'},
              '5':{'j','k','l'},
              '6':{'m','n','o'},
              '7':{'p','q','r','s'},
              '8':{'t','u','v'},
              '9':{'w','x','y','z'},
              '0':{' '}}
        
        ans = []
        
        def dfs(index, tmp):
            if index == len(digits):
                ans.append(tmp)
                return 
            
            for ch in ht[digits[index]]:
                dfs(index+1, tmp + ch)
        
        dfs(0, '')
        return ans 

### 018 4Sum

In [ ]:
class Solution:
    def fourSum(self, nums: List[int], target: int) -> List[List[int]]:
        def findNSum(nums, target, N, tmp, res):
            
            if N < 2 or len(nums) < N or target > N * nums[-1] or target < N * nums[0]:
                return # early termination
            
            elif N == 2:
                left, right = 0, len(nums) - 1 
                while left < right:
                    s = nums[left] + nums[right]
                    if s == target:
                        res.append(tmp + [nums[left], nums[right]])
                        left += 1
                        while left < right and nums[left] == nums[left-1]:
                            left += 1 
                    
                    elif s < target:
                        left += 1
                    else:
                        right -= 1
            
            else:
                for i in range(len(nums)-N+1):
                    if i == 0 or (nums[i-1] != nums[i]):
                        findNSum(nums[i+1:], target - nums[i], N-1, tmp + [nums[i]], res)
        nums.sort()
        res = [] 
        findNSum(nums, target, 4, [], res)
        return res 

### 019 Remove Nth Node From End of List

In [ ]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution:
    def removeNthFromEnd(self, head: ListNode, n: int) -> ListNode:
        dummy = ListNode(-1)
        dummy.next = head
        slow = dummy # stop a n+1 position of end
        fast = dummy
        count = 0
        while count < n:
            fast = fast.next
            count += 1

        while fast.next:
            fast = fast.next
            slow = slow.next

        slow.next = slow.next.next
        return dummy.next

#### Linked List 小结

### 020 Valid Parentheses

In [ ]:
class Solution:
    def isValid(self, s: str) -> bool:
        stack = []
        chmap = {'(': ')', '{':'}', '[':']'}

        for ch in s:
            if ch in chmap:
                stack.append(ch)
            elif ch in chmap.values():
                if stack and chmap[stack[-1]] == ch:
                    stack.pop()
                else:
                    return False
            else:
                return False

        return len(stack) == 0

if __name__ == "__main__":
    s = Solution()
    print(s.isValid(']'))

### 359 Logger Rate Limiter
Easy

Design a logger system that receive stream of messages along with its timestamps, each message should be printed if and only if it is not printed in the last 10 seconds.

Given a message and a timestamp (in seconds granularity), return true if the message should be printed in the given timestamp, otherwise returns false.

It is possible that several messages arrive roughly at the same time.

Example:

Logger logger = new Logger();

// logging string "foo" at timestamp 1
logger.shouldPrintMessage(1, "foo"); returns true; 

// logging string "bar" at timestamp 2
logger.shouldPrintMessage(2,"bar"); returns true;

// logging string "foo" at timestamp 3
logger.shouldPrintMessage(3,"foo"); returns false;

// logging string "bar" at timestamp 8
logger.shouldPrintMessage(8,"bar"); returns false;

// logging string "foo" at timestamp 10
logger.shouldPrintMessage(10,"foo"); returns false;

// logging string "foo" at timestamp 11
logger.shouldPrintMessage(11,"foo"); returns true;

In [ ]:
class Logger:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.lasttime = -10
        self.table = {}
        

    def shouldPrintMessage(self, timestamp: int, message: str) -> bool:
        """
        Returns true if the message should be printed in the given timestamp, otherwise returns false.
        If this method returns false, the message will not be printed.
        The timestamp is in seconds granularity.
        """
        #two message could arrive at the same time. 
        if message in self.table and timestamp - self.table[message] < 10:
            return False
        self.table[message] = timestamp
        return True
        


# Your Logger object will be instantiated and called as such:
# obj = Logger()
# param_1 = obj.shouldPrintMessage(timestamp,message)

### 524 Longest Word in Dictionary through Deleting
Given a string and a string dictionary, find the longest string in the dictionary that can be formed by deleting some characters of the given string. If there are more than one possible results, return the longest word with the smallest lexicographical order. If there is no possible result, return the empty string.

Example 1:
Input:
s = "abpcplea", d = ["ale","apple","monkey","plea"]

Output: 
"apple"
Example 2:
Input:
s = "abpcplea", d = ["a","b","c"]

Output: 
"a"
Note:
All the strings in the input will only contain lower-case letters.
The size of the dictionary won't exceed 1,000.
The length of all the strings in the input won't exceed 1,000.

In [ ]:
class Solution:
    def findLongestWord(self, s: str, d: List[str]) -> str:

        d.sort(key = lambda x: (-len(x), x))
        for word in d:
            i = 0
            for c in s:
                if i < len(word) and word[i] == c:
                    i += 1
            if i == len(word):
                return word
        return ""

In [ ]:
class Solution:
    def findLongestWord(self, s: str, d: List[str]) -> str:

        def check(s, w):
            if len(s) < len(w):
                return False
            i = j = 0
            while i < len(s) and j < len(w):
                if s[i] == w[j]:
                    i += 1 
                    j += 1
                else:
                    while i < len(s) and s[i] != w[j]:
                        i += 1
            return j == len(w)
        
        ans = ''
        
        for word in d:
            if check(s, word):
                if len(word) > len(ans):
                    ans = word
                elif len(word) == len(ans):
                    ans = min(ans, word)
        return ans

### 809. Expressive Words
Medium
Sometimes people repeat letters to represent extra feeling, such as "hello" -> "heeellooo", "hi" -> "hiiii".  In these strings like "heeellooo", we have groups of adjacent letters that are all the same:  "h", "eee", "ll", "ooo".

For some given string S, a query word is stretchy if it can be made to be equal to S by any number of applications of the following extension operation: choose a group consisting of characters c, and add some number of characters c to the group so that the size of the group is 3 or more.

For example, starting with "hello", we could do an extension on the group "o" to get "hellooo", but we cannot get "helloo" since the group "oo" has size less than 3.  Also, we could do another extension like "ll" -> "lllll" to get "helllllooo".  If S = "helllllooo", then the query word "hello" would be stretchy because of these two extension operations: query = "hello" -> "hellooo" -> "helllllooo" = S.

Given a list of query words, return the number of words that are stretchy. 

 

Example:
Input: 
S = "heeellooo"
words = ["hello", "hi", "helo"]
Output: 1
Explanation: 
We can extend "e" and "o" in the word "hello" to get "heeellooo".
We can't extend "helo" to get "heeellooo" because the group "ll" is not size 3 or more.

In [ ]:
class Solution:
    def expressiveWords(self, S: str, words: List[str]) -> int:
        
        def isStrechy(s, t): # source match to target(strechy)
            i = j = 0
            while i < len(s) and j < len(t):
                
                if s[i] != t[j]:
                    return False # not match on this group 
                cur = t[j]
                cntT = 0
                while j < len(t) and t[j] == cur:
                    j += 1 
                    cntT += 1 # count how many cur char in current group
                cntS = 0
                while i < len(s) and s[i] == cur:
                    i += 1
                    cntS += 1 
                if cntS > cntT or (cntT > cntS and cntT < 3):
                    return False # cannot extend          
            return i == len(s) and j == len(t)
        
        ans = 0 
        for word in words:
            ans += isStrechy(word, S)
        return ans

### 844 Backspace String compare

Given two strings S and T, return if they are equal when both are typed into empty text editors. # means a backspace character.
```
Example 1:

Input: S = "ab#c", T = "ad#c"
Output: true
Explanation: Both S and T become "ac".
Example 2:

Input: S = "ab##", T = "c#d#"
Output: true
Explanation: Both S and T become "".
```

In [ ]:
class Solution:
    def backspaceCompare(self, S: str, T: str) -> bool:
        stackS, stackT = [], []
        for ch in S:
            if ch == '#':
                stackS and stackS.pop()
            else:
                stackS.append(ch)
        for ch in T:
            if ch == '#':
                stackT and stackT.pop()
            else:
                stackT.append(ch)
#        print(stackS, stackT)
        return stackS == stackT

In [ ]:
#O(n) time, O(1) space 
class Solution:
    def backspaceCompare(self, S: str, T: str) -> bool:
        i, j = len(S)-1, len(T)-1
        backS = backT = 0
        while True:
            while i >= 0 and (backS or S[i] == '#'):
                backS += 1 if S[i] == '#' else -1
                i -= 1
            while j >= 0 and (backT or T[j] == '#'):
                backT += 1 if T[j] == '#' else -1 
                j -= 1
            
            if not (i>=0 and j>=0 and S[i] == T[j]):
                return i==j==-1
            i -= 1
            j -= 1

### 1007 Minimum Domino Rotations For Equal Row
Medium

In a row of dominoes, A[i] and B[i] represent the top and bottom halves of the i-th domino.  (A domino is a tile with two numbers from 1 to 6 - one on each half of the tile.)

We may rotate the i-th domino, so that A[i] and B[i] swap values.

Return the minimum number of rotations so that all the values in A are the same, or all the values in B are the same.

If it cannot be done, return -1.

 

Example 1:



Input: A = [2,1,2,4,2,2], B = [5,2,6,2,3,2]
Output: 2
Explanation: 
The first figure represents the dominoes as given by A and B: before we do any rotations.
If we rotate the second and fourth dominoes, we can make every value in the top row equal to 2, as indicated by the second figure.
Example 2:

Input: A = [3,5,1,2,3], B = [3,6,3,3,4]
Output: -1
Explanation: 
In this case, it is not possible to rotate the dominoes to make one row of values equal.

In [ ]:
class Solution:
    def minDominoRotations(self, A: List[int], B: List[int]) -> int:
        for x in range(1,7):
            if all(x == a or x == b for a, b in zip (A,B)):
                return min(len(A) - A.count(x), len(B) - B.count(x))
        return -1

### 1096 Brace Expansion II
Hard

Under a grammar given below, strings can represent a set of lowercase words.  Let's use R(expr) to denote the set of words the expression represents.

Grammar can best be understood through simple examples:

Single letters represent a singleton set containing that word.
R("a") = {"a"}
R("w") = {"w"}
When we take a comma delimited list of 2 or more expressions, we take the union of possibilities.
R("{a,b,c}") = {"a","b","c"}
R("{{a,b},{b,c}}") = {"a","b","c"} (notice the final set only contains each word at most once)
When we concatenate two expressions, we take the set of possible concatenations between two words where the first word comes from the first expression and the second word comes from the second expression.
R("{a,b}{c,d}") = {"ac","ad","bc","bd"}
R("a{b,c}{d,e}f{g,h}") = {"abdfg", "abdfh", "abefg", "abefh", "acdfg", "acdfh", "acefg", "acefh"}
Formally, the 3 rules for our grammar:

For every lowercase letter x, we have R(x) = {x}
For expressions e_1, e_2, ... , e_k with k >= 2, we have R({e_1,e_2,...}) = R(e_1) ∪ R(e_2) ∪ ...
For expressions e_1 and e_2, we have R(e_1 + e_2) = {a + b for (a, b) in R(e_1) × R(e_2)}, where + denotes concatenation, and × denotes the cartesian product.
Given an expression representing a set of words under the given grammar, return the sorted list of words that the expression represents.

 
```
Example 1:

Input: "{a,b}{c,{d,e}}"
Output: ["ac","ad","ae","bc","bd","be"]
Example 2:

Input: "{{a,z},a{b,c},{ab,z}}"
Output: ["a","ab","ac","z"]
Explanation: Each distinct word is written only once in the final answer.
 

Constraints:

1 <= expression.length <= 50
expression[i] consists of '{', '}', ','or lowercase English letters.
The given expression represents a set of words based on the grammar given in the description.
```

In [ ]:
class Solution:
    def braceExpansionII(self, expression: str) -> List[str]:
        groups = [[]]
        level = 0
        for i, c in enumerate(expression):
            if c == '{':
                if level == 0:
                    start = i+1
                level += 1
            elif c == '}':
                level -= 1
                if level == 0:
                    groups[-1].append(self.braceExpansionII(expression[start:i]))
            elif c == ',' and level == 0:
                groups.append([])
            elif level == 0:
                groups[-1].append([c])
        word_set = set()
        for group in groups:
            word_set |= set(map(''.join, itertools.product(*group)))
        return sorted(word_set)

In [1]:
def product(*args, **kwds):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = map(tuple, args) * kwds.get('repeat', 1)
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)

## Onsite

### Self-Describing Number

In [20]:
def isSelfDescribing(n):
    s = str(n)
    return all(s.count(str(i)) == int(ch) for i, ch in enumerate(s))
[x for x in range(400000) if isSelfDescribing(x)]

[1210, 2020, 21200]

In [22]:
def impl(d, c, m):
    if m < 0: return
    if d == c[:len(d)]: print(d)
    for i in range(c[len(d)],m+1):
        dd = d+[i]
        if i<len(dd) and c[i]==dd[i]: continue
        impl(dd,c[:i]+[c[i]+1]+c[i+1:],m-i)
        
def self(n): impl([], [0]*(n+1), n)

self(10)

[]
[1, 2, 1, 0]
[2, 0, 2, 0]
[2, 1, 2, 0, 0]
[3, 2, 1, 1, 0, 0, 0]
[4, 2, 1, 0, 1, 0, 0, 0]
[5, 2, 1, 0, 0, 1, 0, 0, 0]
[6, 2, 1, 0, 0, 0, 1, 0, 0, 0]


### Split the given array into K sub-arrays such that maximum sum of all sub arrays is minimum

题目是想象一条直线上有n个点，要求在线上选取k个点，使得这n个点到k个点的距离最短，比如n=4，k=2，n个点分别在1，3，100，103，那k点应该取2，102。拿到一看感觉挺棘手，楼主就说了一个大概的思路，大概就是把n个数split成k个连续子数组，子数组内选一个点到子数组内所有元素距离相加最短

In [ ]:
# Function to check if mid can
# be maximum sub – arrays sum
    def check(mid, array, n, K):
    count = 0
    sum = 0
    for i in range(n):

    # If individual element is greater
    # maximum possible sum
    if (array[i] > mid):
    return False

    # Increase sum of current sub – array
    sum += array[i]

    # If the sum is greater than
    # mid increase count
    if (sum > mid):
    count += 1
    sum = array[i]
    count += 1

    # Check condition
    if (count <= K): 
        return True 
    return False 


# Function to find maximum subarray sum # which is minimum def solve(array, n, K): start = 1 end = 0 for i in range(n): end += array[i] # Answer stores possible # maximum sub array sum answer = 0 while (start <= end): mid = (start + end) // 2 # If mid is possible solution # Put answer = mid; if (check(mid, array, n, K)): answer = mid end = mid - 1 else: start = mid + 1 return answer # Driver Code if __name__ == '__main__': array = [1, 2, 3, 4] n = len(array) K = 3 print(solve(array, n, K)) # This code is contributed by # Surendra_Gangwar [tabby title="C#"]